In [1]:
%matplotlib inline


Word2Vec Model
==============

Introduces Gensim's Word2Vec model and demonstrates its use on the Lee Corpus.




In [24]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In case you missed the buzz, word2vec is a widely featured as a member of the
“new wave” of machine learning algorithms based on neural networks, commonly
referred to as "deep learning" (though word2vec itself is rather shallow).
Using large amounts of unannotated plain text, word2vec learns relationships
between words automatically. The output are vectors, one vector per word,
with remarkable linear relationships that allow us to do things like:

* vec("king") - vec("man") + vec("woman") =~ vec("queen")
* vec("Montreal Canadiens") – vec("Montreal") + vec("Toronto") =~ vec("Toronto Maple Leafs").

Word2vec is very useful in `automatic text tagging
<https://github.com/RaRe-Technologies/movie-plots-by-genre>`_\ , recommender
systems and machine translation.

This tutorial:

#. Introduces ``Word2Vec`` as an improvement over traditional bag-of-words
#. Shows off a demo of ``Word2Vec`` using a pre-trained model
#. Demonstrates training a new model from your own data
#. Demonstrates loading and saving models
#. Introduces several training parameters and demonstrates their effect
#. Discusses memory requirements
#. Visualizes Word2Vec embeddings by applying dimensionality reduction

Review: Bag-of-words
--------------------

.. Note:: Feel free to skip these review sections if you're already familiar with the models.

You may be familiar with the `bag-of-words model
<https://en.wikipedia.org/wiki/Bag-of-words_model>`_ from the
`core_concepts_vector` section.
This model transforms each document to a fixed-length vector of integers.
For example, given the sentences:

- ``John likes to watch movies. Mary likes movies too.``
- ``John also likes to watch football games. Mary hates football.``

The model outputs the vectors:

- ``[1, 2, 1, 1, 2, 1, 1, 0, 0, 0, 0]``
- ``[1, 1, 1, 1, 0, 1, 0, 1, 2, 1, 1]``

Each vector has 10 elements, where each element counts the number of times a
particular word occurred in the document.
The order of elements is arbitrary.
In the example above, the order of the elements corresponds to the words:
``["John", "likes", "to", "watch", "movies", "Mary", "too", "also", "football", "games", "hates"]``.

Bag-of-words models are surprisingly effective, but have several weaknesses.

First, they lose all information about word order: "John likes Mary" and
"Mary likes John" correspond to identical vectors. There is a solution: bag
of `n-grams <https://en.wikipedia.org/wiki/N-gram>`__
models consider word phrases of length n to represent documents as
fixed-length vectors to capture local word order but suffer from data
sparsity and high dimensionality.

Second, the model does not attempt to learn the meaning of the underlying
words, and as a consequence, the distance between vectors doesn't always
reflect the difference in meaning.  The ``Word2Vec`` model addresses this
second problem.

Introducing: the ``Word2Vec`` Model
-----------------------------------

``Word2Vec`` is a more recent model that embeds words in a lower-dimensional
vector space using a shallow neural network. The result is a set of
word-vectors where vectors close together in vector space have similar
meanings based on context, and word-vectors distant to each other have
differing meanings. For example, ``strong`` and ``powerful`` would be close
together and ``strong`` and ``Paris`` would be relatively far.

The are two versions of this model and :py:class:`~gensim.models.word2vec.Word2Vec`
class implements them both:

1. Skip-grams (SG)
2. Continuous-bag-of-words (CBOW)

.. Important::
  Don't let the implementation details below scare you.
  They're advanced material: if it's too much, then move on to the next section.

The `Word2Vec Skip-gram <http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model>`__
model, for example, takes in pairs (word1, word2) generated by moving a
window across text data, and trains a 1-hidden-layer neural network based on
the synthetic task of given an input word, giving us a predicted probability
distribution of nearby words to the input. A virtual `one-hot
<https://en.wikipedia.org/wiki/One-hot>`__ encoding of words
goes through a 'projection layer' to the hidden layer; these projection
weights are later interpreted as the word embeddings. So if the hidden layer
has 300 neurons, this network will give us 300-dimensional word embeddings.

Continuous-bag-of-words Word2vec is very similar to the skip-gram model. It
is also a 1-hidden-layer neural network. The synthetic training task now uses
the average of multiple input context words, rather than a single word as in
skip-gram, to predict the center word. Again, the projection weights that
turn one-hot words into averageable vectors, of the same width as the hidden
layer, are interpreted as the word embeddings.




Word2Vec Demo
-------------

To see what ``Word2Vec`` can do, let's download a pre-trained model and play
around with it. We will fetch the Word2Vec model trained on part of the
Google News dataset, covering approximately 3 million words and phrases. Such
a model can take hours to train, but since it's already available,
downloading and loading it with Gensim takes minutes.

.. Important::
  The model is approximately 2GB, so you'll need a decent network connection
  to proceed.  Otherwise, skip ahead to the "Training Your Own Model" section
  below.

You may also check out an `online word2vec demo
<http://radimrehurek.com/2014/02/word2vec-tutorial/#app>`_ where you can try
this vector algebra for yourself. That demo runs ``word2vec`` on the
**entire** Google News dataset, of **about 100 billion words**.




In [25]:
import gensim.models

wv=gensim.models.KeyedVectors.load_word2vec_format('../data/GoogleNews-vectors-negative300.bin.gz', binary=True)

2020-02-03 23:11:00,017 : INFO : loading projection weights from ../data/GoogleNews-vectors-negative300.bin.gz
2020-02-03 23:13:32,331 : INFO : loaded (3000000, 300) matrix from ../data/GoogleNews-vectors-negative300.bin.gz


A common operation is to retrieve the vocabulary of a model.  That is trivial:



In [26]:
for i, word in enumerate(wv.vocab):
    if i == 10:
        break
    print(word)

</s>
in
for
that
is
on
##
The
with
said


We can easily obtain vectors for terms the model is familiar with:




In [6]:
vec_king = wv['king']

可惜，这个model不能推断它不知道的词。这是Word2Vec其中一个限制：如果你很在意这个限制，请使用 FastText 模型。

In [7]:
try:
    vec_cameroon = wv['cameroon']
except KeyError:
    print("The word 'cameroon' does not appear in this model")

The word 'cameroon' does not appear in this model


Moving on, ``Word2Vec`` supports several word similarity tasks out of the
box.  You can see how the similarity intuitively decreases as the words get
less and less similar.




In [9]:
pairs = [
    ('car', 'minivan'),   # a minivan is a kind of car
    ('car', 'bicycle'),   # still a wheeled vehicle
    ('car', 'airplane'),  # ok, no wheels, but still a vehicle
    ('car', 'cereal'),    # ... and so on
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

'car'	'minivan'	0.69
'car'	'bicycle'	0.54
'car'	'airplane'	0.42
'car'	'cereal'	0.14
'car'	'communism'	0.06


Print the 5 most similar words to "car" or "minivan"



In [10]:
print(wv.most_similar(positive=['car', 'minivan'], topn=5))

2020-02-03 22:46:02,729 : INFO : precomputing L2-norms of word weight vectors


[('SUV', 0.8532191514968872), ('vehicle', 0.8175784349441528), ('pickup_truck', 0.7763689160346985), ('Jeep', 0.7567334175109863), ('Ford_Explorer', 0.7565719485282898)]


Which of the below does not belong in the sequence?



In [11]:
print(wv.doesnt_match(['fire', 'water', 'land', 'sea', 'air', 'car']))

car


/Users/liuning11/.conda/envs/myfaiss/lib/python3.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


Training Your Own Model
-----------------------

To start, you'll need some data for training the model.  For the following
examples, we'll use the `Lee Corpus
<https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/test/test_data/lee_background.cor>`_
(which you already have if you've installed gensim).

该语料很小，能完全放在内存中，但我们仍然实现一个针对内存友好的迭代器，按行读取，以便处理大规模语料。

In [12]:
from gensim.test.utils import datapath
from gensim import utils

corpus_path='../data/lee_background.cor'

class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""

    def __iter__(self):
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

2020-02-03 22:49:22,016 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-02-03 22:49:22,020 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


If we wanted to do any custom preprocessing, e.g. decode a non-standard
encoding, lowercase, remove numbers, extract named entities... All of this can
be done inside the ``MyCorpus`` iterator and ``word2vec`` doesn’t need to
know. All that is required is that the input yields one sentence (list of
utf8 words) after another.

Let's go ahead and train a model on our corpus.  Don't worry about the
training parameters much for now, we'll revisit them later.




In [13]:
import gensim.models

sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences)

2020-02-03 22:49:28,784 : INFO : collecting all words and their counts
2020-02-03 22:49:28,790 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-02-03 22:49:28,885 : INFO : collected 6981 word types from a corpus of 58152 raw words and 300 sentences
2020-02-03 22:49:28,886 : INFO : Loading a fresh vocabulary
2020-02-03 22:49:28,918 : INFO : effective_min_count=5 retains 1750 unique words (25% of original 6981, drops 5231)
2020-02-03 22:49:28,919 : INFO : effective_min_count=5 leaves 49335 word corpus (84% of original 58152, drops 8817)
2020-02-03 22:49:28,925 : INFO : deleting the raw counts dictionary of 6981 items
2020-02-03 22:49:28,928 : INFO : sample=0.001 downsamples 51 most-common words
2020-02-03 22:49:28,930 : INFO : downsampling leaves estimated 35935 word corpus (72.8% of prior 49335)
2020-02-03 22:49:28,936 : INFO : estimated required memory for 1750 words and 100 dimensions: 2275000 bytes
2020-02-03 22:49:28,937 : INFO : resetting layer weight

Once we have our model, we can use it in the same way as in the demo above.

The main part of the model is ``model.wv``\ , where "wv" stands for "word vectors".




In [14]:
vec_king = model.wv['king'] # dim=100

Retrieving the vocabulary works the same way:



In [15]:
for i, word in enumerate(model.wv.vocab):
    if i == 10:
        break
    print(word)

hundreds
of
people
have
been
forced
to
their
homes
in


Storing and loading models
--------------------------

You'll notice that training non-trivial models can take time.  Once you've
trained your model and it works as expected, you can save it to disk.  That
way, you don't have to spend time training it all over again later.

You can store/load models using the standard gensim methods:




In [16]:
import tempfile

with tempfile.NamedTemporaryFile(prefix='gensim-model-', delete=False) as tmp:
    temporary_filepath = tmp.name
    model.save(temporary_filepath)
    #
    # The model is now safely stored in the filepath.
    # You can copy it to other machines, share it with others, etc.
    #
    # To load a saved model:
    #
    new_model = gensim.models.Word2Vec.load(temporary_filepath)

2020-02-03 22:50:01,465 : INFO : saving Word2Vec object under /var/folders/fb/jcf536t9265fc_nlmqdcwd9jsf5jrm/T/gensim-model-wiazqwrx, separately None
2020-02-03 22:50:01,466 : INFO : not storing attribute vectors_norm
2020-02-03 22:50:01,467 : INFO : not storing attribute cum_table
2020-02-03 22:50:01,491 : INFO : saved /var/folders/fb/jcf536t9265fc_nlmqdcwd9jsf5jrm/T/gensim-model-wiazqwrx
2020-02-03 22:50:01,492 : INFO : loading Word2Vec object from /var/folders/fb/jcf536t9265fc_nlmqdcwd9jsf5jrm/T/gensim-model-wiazqwrx
2020-02-03 22:50:01,509 : INFO : loading wv recursively from /var/folders/fb/jcf536t9265fc_nlmqdcwd9jsf5jrm/T/gensim-model-wiazqwrx.wv.* with mmap=None
2020-02-03 22:50:01,510 : INFO : setting ignored attribute vectors_norm to None
2020-02-03 22:50:01,511 : INFO : loading vocabulary recursively from /var/folders/fb/jcf536t9265fc_nlmqdcwd9jsf5jrm/T/gensim-model-wiazqwrx.vocabulary.* with mmap=None
2020-02-03 22:50:01,511 : INFO : loading trainables recursively from /var/

which uses pickle internally, optionally ``mmap``\ ‘ing the model’s internal
large NumPy matrices into virtual memory directly from disk files, for
inter-process memory sharing.

In addition, you can load models created by the original C tool, both using
its text and binary formats::

  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.txt', binary=False)
  # using gzipped/bz2 input works too, no need to unzip
  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.bin.gz', binary=True)




Training Parameters
-------------------

``Word2Vec`` accepts several parameters that affect both training speed and quality.

min_count
---------

``min_count`` is for pruning the internal dictionary. Words that appear only
once or twice in a billion-word corpus are probably uninteresting typos and
garbage. In addition, there’s not enough data to make any meaningful training
on those words, so it’s best to ignore them:

default value of min_count=5



In [ ]:
model = gensim.models.Word2Vec(sentences, min_count=10)

size
----

``size`` is the number of dimensions (N) of the N-dimensional space that
gensim Word2Vec maps the words onto.

Bigger size values require more training data, but can lead to better (more
accurate) models. Reasonable values are in the tens to hundreds.




In [ ]:
# default value of size=100
model = gensim.models.Word2Vec(sentences, size=200)

workers
-------

``workers`` , the last of the major parameters (full list `here
<http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec>`_)
is for training parallelization, to speed up training:




In [ ]:
# default value of workers=3 (tutorial says 1...)
model = gensim.models.Word2Vec(sentences, workers=4)

The ``workers`` parameter only has an effect if you have `Cython
<http://cython.org/>`_ installed. Without Cython, you’ll only be able to use
one core because of the `GIL
<https://wiki.python.org/moin/GlobalInterpreterLock>`_ (and ``word2vec``
training will be `miserably slow
<http://rare-technologies.com/word2vec-in-python-part-two-optimizing/>`_\ ).




Memory
------

At its core, ``word2vec`` model parameters are stored as matrices (NumPy
arrays). Each array is **#vocabulary** (controlled by min_count parameter)
times **#size** (size parameter) of floats (single precision aka 4 bytes).

Three such matrices are held in RAM (work is underway to reduce that number
to two, or even one). So if your input contains 100,000 unique words, and you
asked for layer ``size=200``\ , the model will require approx.
``100,000*200*4*3 bytes = ~229MB``.

There’s a little extra memory needed for storing the vocabulary tree (100,000 words would take a few megabytes), but unless your words are extremely loooong strings, memory footprint will be dominated by the three matrices above.




Evaluating
----------

``Word2Vec`` training is an unsupervised task, there’s no good way to
objectively evaluate the result. Evaluation depends on your end application.

Google has released their testing set of about 20,000 syntactic and semantic
test examples, following the “A is to B as C is to D” task. It is provided in
the 'datasets' folder.

For example a syntactic analogy of comparative type is bad:worse;good:?.
There are total of 9 types of syntactic comparisons in the dataset like
plural nouns and nouns of opposite meaning.

The semantic questions contain five types of semantic analogies, such as
capital cities (Paris:France;Tokyo:?) or family members
(brother:sister;dad:?).




Gensim supports the same evaluation set, in exactly the same format:




In [ ]:
model.accuracy('../data/questions-words.txt')

This ``accuracy`` takes an `optional parameter
<http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec.accuracy>`_
``restrict_vocab`` which limits which test examples are to be considered.




In the December 2016 release of Gensim we added a better way to evaluate semantic similarity.

By default it uses an academic dataset WS-353 but one can create a dataset
specific to your business based on it. It contains word pairs together with
human-assigned similarity judgments. It measures the relatedness or
co-occurrence of two words. For example, 'coast' and 'shore' are very similar
as they appear in the same context. At the same time 'clothes' and 'closet'
are less similar because they are related but not interchangeable.




In [ ]:
model.evaluate_word_pairs('../data/wordsim353.tsv')

.. Important::
  Good performance on Google's or WS-353 test set doesn’t mean word2vec will
  work well in your application, or vice versa. It’s always best to evaluate
  directly on your intended task. For an example of how to use word2vec in a
  classifier pipeline, see this `tutorial
  <https://github.com/RaRe-Technologies/movie-plots-by-genre>`_.




Online training / Resuming training
-----------------------------------

Advanced users can load a model and continue training it with more sentences
and `new vocabulary words <online_w2v_tutorial.ipynb>`_:




In [ ]:
model = gensim.models.Word2Vec.load(temporary_filepath)
more_sentences = [
    ['Advanced', 'users', 'can', 'load', 'a', 'model',
     'and', 'continue', 'training', 'it', 'with', 'more', 'sentences']
]
model.build_vocab(more_sentences, update=True)
model.train(more_sentences, total_examples=model.corpus_count, epochs=model.iter)

# cleaning up temporary file
import os
os.remove(temporary_filepath)

You may need to tweak the ``total_words`` parameter to ``train()``,
depending on what learning rate decay you want to simulate.

Note that it’s not possible to resume training with models generated by the C
tool, ``KeyedVectors.load_word2vec_format()``. You can still use them for
querying/similarity, but information vital for training (the vocab tree) is
missing there.




Training Loss Computation
-------------------------

The parameter ``compute_loss`` can be used to toggle computation of loss
while training the Word2Vec model. The computed loss is stored in the model
attribute ``running_training_loss`` and can be retrieved using the function
``get_latest_training_loss`` as follows :




In [ ]:
# instantiating and training the Word2Vec model
model_with_loss = gensim.models.Word2Vec(
    sentences,
    min_count=1,
    compute_loss=True,
    hs=0,
    sg=1,
    seed=42
)

# getting the training loss value
training_loss = model_with_loss.get_latest_training_loss()
print(training_loss)

Benchmarks
----------

Let's run some benchmarks to see effect of the training loss computation code
on training time.

We'll use the following data for the benchmarks:

#. Lee Background corpus: included in gensim's test data
#. Text8 corpus.  To demonstrate the effect of corpus size, we'll look at the
   first 1MB, 10MB, 50MB of the corpus, as well as the entire thing.




In [17]:
import io
import os

import gensim.models.word2vec
import gensim.downloader as api
import smart_open
import logging

def head(path, size):
    with smart_open.open(path) as fin:
        return io.StringIO(fin.read(size))


def generate_input_data():
    lee_path = '../data/lee_background.cor'
    ls = gensim.models.word2vec.LineSentence(lee_path)
    ls.name = '25kB'
    yield ls

    text8_path = '../data/text8'
    labels = ('1MB', '10MB', '50MB', '100MB')
    sizes = (1024 ** 2, 10 * 1024 ** 2, 50 * 1024 ** 2, 100 * 1024 ** 2)
    for l, s in zip(labels, sizes):
        ls = gensim.models.word2vec.LineSentence(head(text8_path, s))
        ls.name = l
        yield ls


input_data = list(generate_input_data())

We now compare the training time taken for different combinations of input
data and model training parameters like ``hs`` and ``sg``.

For each combination, we repeat the test several times to obtain the mean and
standard deviation of the test duration.




In [18]:
# Temporarily reduce logging verbosity
logging.root.level = logging.ERROR

import time
import numpy as np
import pandas as pd

train_time_values = []
seed_val = 42
sg_values = [0, 1]
hs_values = [0, 1]

fast = True
if fast:
    input_data_subset = input_data[:3]
else:
    input_data_subset = input_data


for data in input_data_subset:
    for sg_val in sg_values:
        for hs_val in hs_values:
            for loss_flag in [True, False]:
                time_taken_list = []
                for i in range(3):
                    start_time = time.time()
                    w2v_model = gensim.models.Word2Vec(
                        data,
                        compute_loss=loss_flag,
                        sg=sg_val,
                        hs=hs_val,
                        seed=seed_val,
                    )
                    time_taken_list.append(time.time() - start_time)

                time_taken_list = np.array(time_taken_list)
                time_mean = np.mean(time_taken_list)
                time_std = np.std(time_taken_list)

                model_result = {
                    'train_data': data.name,
                    'compute_loss': loss_flag,
                    'sg': sg_val,
                    'hs': hs_val,
                    'train_time_mean': time_mean,
                    'train_time_std': time_std,
                }
                print("Word2vec model #%i: %s" % (len(train_time_values), model_result))
                train_time_values.append(model_result)

train_times_table = pd.DataFrame(train_time_values)
train_times_table = train_times_table.sort_values(
    by=['train_data', 'sg', 'hs', 'compute_loss'],
    ascending=[False, False, True, False],
)
print(train_times_table)

2020-02-03 22:54:28,575 : INFO : collecting all words and their counts
2020-02-03 22:54:28,577 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-02-03 22:54:28,598 : INFO : collected 10781 word types from a corpus of 59890 raw words and 300 sentences
2020-02-03 22:54:28,599 : INFO : Loading a fresh vocabulary
2020-02-03 22:54:28,606 : INFO : effective_min_count=5 retains 1762 unique words (16% of original 10781, drops 9019)
2020-02-03 22:54:28,606 : INFO : effective_min_count=5 leaves 46084 word corpus (76% of original 59890, drops 13806)
2020-02-03 22:54:28,613 : INFO : deleting the raw counts dictionary of 10781 items
2020-02-03 22:54:28,614 : INFO : sample=0.001 downsamples 45 most-common words
2020-02-03 22:54:28,614 : INFO : downsampling leaves estimated 32610 word corpus (70.8% of prior 46084)
2020-02-03 22:54:28,620 : INFO : estimated required memory for 1762 words and 100 dimensions: 2290600 bytes
2020-02-03 22:54:28,621 : INFO : resetting layer we

2020-02-03 22:54:30,317 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-02-03 22:54:30,321 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-02-03 22:54:30,321 : INFO : EPOCH - 2 : training on 59890 raw words (32552 effective words) took 0.0s, 945396 effective words/s
2020-02-03 22:54:30,351 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-02-03 22:54:30,355 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-02-03 22:54:30,358 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-02-03 22:54:30,359 : INFO : EPOCH - 3 : training on 59890 raw words (32517 effective words) took 0.0s, 909672 effective words/s
2020-02-03 22:54:30,391 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-02-03 22:54:30,394 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-02-03 22:54:30,398 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-0

Word2vec model #0: {'train_data': '25kB', 'compute_loss': True, 'sg': 0, 'hs': 0, 'train_time_mean': 0.6225600242614746, 'train_time_std': 0.008212428504875411}


2020-02-03 22:54:30,882 : INFO : training model with 3 workers on 1762 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2020-02-03 22:54:30,913 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-02-03 22:54:30,914 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-02-03 22:54:30,920 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-02-03 22:54:30,921 : INFO : EPOCH - 1 : training on 59890 raw words (32543 effective words) took 0.0s, 868729 effective words/s
2020-02-03 22:54:30,957 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-02-03 22:54:30,960 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-02-03 22:54:30,964 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-02-03 22:54:30,964 : INFO : EPOCH - 2 : training on 59890 raw words (32552 effective words) took 0.0s, 798707 effective words/s
2020-02-03 22:54:30,996 : INFO : worker

2020-02-03 22:54:32,268 : INFO : EPOCH - 4 : training on 59890 raw words (32769 effective words) took 0.0s, 1001040 effective words/s
2020-02-03 22:54:32,299 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-02-03 22:54:32,302 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-02-03 22:54:32,307 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-02-03 22:54:32,308 : INFO : EPOCH - 5 : training on 59890 raw words (32723 effective words) took 0.0s, 878691 effective words/s
2020-02-03 22:54:32,309 : INFO : training on a 299450 raw words (163104 effective words) took 0.2s, 900933 effective words/s
2020-02-03 22:54:32,312 : INFO : collecting all words and their counts
2020-02-03 22:54:32,313 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-02-03 22:54:32,337 : INFO : collected 10781 word types from a corpus of 59890 raw words and 300 sentences
2020-02-03 22:54:32,339 : INFO : Loading a fresh vocab

...

Word2vec model #22: {'train_data': '10MB', 'compute_loss': True, 'sg': 1, 'hs': 1, 'train_time_mean': 42.05006639162699, 'train_time_std': 0.7657778590513281}


2020-02-03 23:03:18,781 : INFO : collected 73167 word types from a corpus of 1788017 raw words and 179 sentences
2020-02-03 23:03:18,782 : INFO : Loading a fresh vocabulary
2020-02-03 23:03:18,839 : INFO : effective_min_count=5 retains 20167 unique words (27% of original 73167, drops 53000)
2020-02-03 23:03:18,840 : INFO : effective_min_count=5 leaves 1703716 word corpus (95% of original 1788017, drops 84301)
2020-02-03 23:03:18,907 : INFO : deleting the raw counts dictionary of 73167 items
2020-02-03 23:03:18,911 : INFO : sample=0.001 downsamples 38 most-common words
2020-02-03 23:03:18,911 : INFO : downsampling leaves estimated 1242287 word corpus (72.9% of prior 1703716)
2020-02-03 23:03:18,936 : INFO : constructing a huffman tree from 20167 words
2020-02-03 23:03:19,538 : INFO : built huffman tree with maximum node depth 18
2020-02-03 23:03:19,576 : INFO : estimated required memory for 20167 words and 100 dimensions: 38317300 bytes
2020-02-03 23:03:19,577 : INFO : resetting layer w

2020-02-03 23:04:05,834 : INFO : EPOCH 1 - PROGRESS: at 26.26% examples, 164977 words/s, in_qsize 5, out_qsize 0
2020-02-03 23:04:06,877 : INFO : EPOCH 1 - PROGRESS: at 42.46% examples, 174495 words/s, in_qsize 5, out_qsize 0
2020-02-03 23:04:07,942 : INFO : EPOCH 1 - PROGRESS: at 59.22% examples, 178742 words/s, in_qsize 5, out_qsize 0
2020-02-03 23:04:08,952 : INFO : EPOCH 1 - PROGRESS: at 75.42% examples, 182796 words/s, in_qsize 5, out_qsize 0
2020-02-03 23:04:09,985 : INFO : EPOCH 1 - PROGRESS: at 91.06% examples, 183781 words/s, in_qsize 5, out_qsize 0
2020-02-03 23:04:10,455 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-02-03 23:04:10,531 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-02-03 23:04:10,544 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-02-03 23:04:10,544 : INFO : EPOCH - 1 : training on 1788017 raw words (1242946 effective words) took 6.7s, 184710 effective words/s
2020-02-03 23:04:11,571 :

2020-02-03 23:04:53,381 : INFO : EPOCH 2 - PROGRESS: at 11.17% examples, 142380 words/s, in_qsize 5, out_qsize 0
2020-02-03 23:04:54,439 : INFO : EPOCH 2 - PROGRESS: at 25.70% examples, 157941 words/s, in_qsize 5, out_qsize 0
2020-02-03 23:04:55,450 : INFO : EPOCH 2 - PROGRESS: at 40.78% examples, 167091 words/s, in_qsize 5, out_qsize 0
2020-02-03 23:04:56,536 : INFO : EPOCH 2 - PROGRESS: at 55.87% examples, 167765 words/s, in_qsize 6, out_qsize 0
2020-02-03 23:04:57,567 : INFO : EPOCH 2 - PROGRESS: at 70.95% examples, 169903 words/s, in_qsize 6, out_qsize 0
2020-02-03 23:04:58,647 : INFO : EPOCH 2 - PROGRESS: at 85.47% examples, 169567 words/s, in_qsize 5, out_qsize 0
2020-02-03 23:04:59,586 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-02-03 23:04:59,658 : INFO : EPOCH 2 - PROGRESS: at 99.44% examples, 169682 words/s, in_qsize 1, out_qsize 1
2020-02-03 23:04:59,659 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-02-03 23:04:59,685 : I

Word2vec model #23: {'train_data': '10MB', 'compute_loss': False, 'sg': 1, 'hs': 1, 'train_time_mean': 40.79019276301066, 'train_time_std': 0.2595709934863263}
   train_data  compute_loss  sg  hs  train_time_mean  train_time_std
4        25kB          True   1   0         0.847283        0.066428
5        25kB         False   1   0         0.765171        0.005743
6        25kB          True   1   1         1.478252        0.204977
7        25kB         False   1   1         2.092017        0.867424
0        25kB          True   0   0         0.622560        0.008212
1        25kB         False   0   0         0.621660        0.028678
2        25kB          True   0   1         0.787105        0.023877
3        25kB         False   0   1         0.773115        0.005728
12        1MB          True   1   0         2.475293        0.265729
13        1MB         False   1   0         2.220606        0.046947
14        1MB          True   1   1         3.995769        0.219305
15        1M

Adding Word2Vec "model to dict" method to production pipeline
-------------------------------------------------------------

Suppose, we still want more performance improvement in production.

One good way is to cache all the similar words in a dictionary.

So that next time when we get the similar query word, we'll search it first in the dict.

And if it's a hit then we will show the result directly from the dictionary.

otherwise we will query the word and then cache it so that it doesn't miss next time.




In [19]:
# re-enable logging
logging.root.level = logging.INFO

most_similars_precalc = {word : model.wv.most_similar(word) for word in model.wv.index2word}
for i, (key, value) in enumerate(most_similars_precalc.items()):
    if i == 3:
        break
    print(key, value)

2020-02-03 23:05:20,711 : INFO : precomputing L2-norms of word weight vectors


the [('on', 0.9999356269836426), ('by', 0.9999175071716309), ('which', 0.9999114274978638), ('also', 0.9999108910560608), ('and', 0.9999102354049683), ('an', 0.9999089241027832), ('for', 0.9999087452888489), ('days', 0.9999085664749146), ('with', 0.9999066591262817), ('australian', 0.9999063014984131)]
to [('also', 0.9999505877494812), ('about', 0.9999481439590454), ('for', 0.9999478459358215), ('from', 0.9999448657035828), ('but', 0.9999440312385559), ('at', 0.9999434947967529), ('company', 0.9999430775642395), ('or', 0.9999391436576843), ('before', 0.9999383687973022), ('who', 0.9999364018440247)]
of [('and', 0.9999475479125977), ('in', 0.9999444484710693), ('on', 0.9999402761459351), ('with', 0.9999359846115112), ('over', 0.9999345541000366), ('which', 0.9999303221702576), ('after', 0.9999282956123352), ('two', 0.9999276995658875), ('an', 0.9999268651008606), ('into', 0.9999268054962158)]


Comparison with and without caching
-----------------------------------

for time being lets take 4 words randomly




In [20]:
import time
words = ['voted', 'few', 'their', 'around']

Without caching




In [21]:
start = time.time()
for word in words:
    result = model.wv.most_similar(word)
    print(result)
end = time.time()
print(end - start)

[('lower', 0.9980545043945312), ('army', 0.9980535507202148), ('unions', 0.9980233907699585), ('first', 0.9980073571205139), ('secretary', 0.9980006217956543), ('israel', 0.997992753982544), ('weather', 0.9979914426803589), ('air', 0.9979841709136963), ('child', 0.997980535030365), ('team', 0.9979622960090637)]
[('were', 0.9997997283935547), ('around', 0.9997900724411011), ('with', 0.9997897744178772), ('three', 0.999785840511322), ('which', 0.9997817873954773), ('for', 0.9997814893722534), ('two', 0.9997813701629639), ('into', 0.9997802376747131), ('now', 0.9997799396514893), ('as', 0.999779462814331)]
[('for', 0.9999517202377319), ('at', 0.9999475479125977), ('with', 0.9999464750289917), ('from', 0.9999454021453857), ('also', 0.9999444484710693), ('but', 0.9999433159828186), ('and', 0.999941349029541), ('his', 0.9999404549598694), ('which', 0.9999392032623291), ('were', 0.9999378323554993)]
[('with', 0.9999535083770752), ('and', 0.9999483823776245), ('for', 0.9999468326568604), ('aft

Now with caching




In [22]:
start = time.time()
for word in words:
    if 'voted' in most_similars_precalc:
        result = most_similars_precalc[word]
        print(result)
    else:
        result = model.wv.most_similar(word)
        most_similars_precalc[word] = result
        print(result)

end = time.time()
print(end - start)

[('lower', 0.9980545043945312), ('army', 0.9980535507202148), ('unions', 0.9980233907699585), ('first', 0.9980073571205139), ('secretary', 0.9980006217956543), ('israel', 0.997992753982544), ('weather', 0.9979914426803589), ('air', 0.9979841709136963), ('child', 0.997980535030365), ('team', 0.9979622960090637)]
[('were', 0.9997997283935547), ('around', 0.9997900724411011), ('with', 0.9997897744178772), ('three', 0.999785840511322), ('which', 0.9997817873954773), ('for', 0.9997814893722534), ('two', 0.9997813701629639), ('into', 0.9997802376747131), ('now', 0.9997799396514893), ('as', 0.999779462814331)]
[('for', 0.9999517202377319), ('at', 0.9999475479125977), ('with', 0.9999464750289917), ('from', 0.9999454021453857), ('also', 0.9999444484710693), ('but', 0.9999433159828186), ('and', 0.999941349029541), ('his', 0.9999404549598694), ('which', 0.9999392032623291), ('were', 0.9999378323554993)]
[('with', 0.9999535083770752), ('and', 0.9999483823776245), ('for', 0.9999468326568604), ('aft

Clearly you can see the improvement but this difference will be even larger
when we take more words in the consideration.




Visualising the Word Embeddings
-------------------------------

The word embeddings made by the model can be visualised by reducing
dimensionality of the words to 2 dimensions using tSNE.

Visualisations can be used to notice semantic and syntactic trends in the data.

Example:

* Semantic: words like cat, dog, cow, etc. have a tendency to lie close by
* Syntactic: words like run, running or cut, cutting lie close together.

Vector relations like vKing - vMan = vQueen - vWoman can also be noticed.

.. Important::
  The model used for the visualisation is trained on a small corpus. Thus
  some of the relations might not be so clear.




In [23]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling


def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    for word in model.wv.vocab:
        vectors.append(model.wv[word])
        labels.append(word)

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)

    # reduce using t-SNE
    vectors = np.asarray(vectors)
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(model)

def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    #
    # Label randomly subsampled 25 data points
    #
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly

plot_function(x_vals, y_vals, labels)

Conclusion
----------

In this tutorial we learned how to train word2vec models on your custom data
and also how to evaluate it. Hope that you too will find this popular tool
useful in your Machine Learning tasks!

Links
-----

- API docs: :py:mod:`gensim.models.word2vec`
- `Original C toolkit and word2vec papers by Google <https://code.google.com/archive/p/word2vec/>`_.


